# Elasticsearch setup

In [1]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

stoplist = open('../HW1/reference/stoplist.txt')
stoplist = [line.strip() for line in stoplist]

OKAPI_BM_TOP1000 = '../HW1/results/Okapi-BM25.txt'
QREL_ADHOC = '../HW1/trec_eval/qrels.adhoc.51-100.AP89.txt'

# Create union set of okapi-bm25 and qrel

In [2]:
def okapi_qrel_union(QREL_ADHOC, OKAPI_BM_TOP1000):
    union_set = {}
    with open (OKAPI_BM_TOP1000, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            items = line.split()
            queryno, docno = int(items[0]), items[2]
            if queryno in union_set:
                union_set[queryno].add(docno)
            else:  
                union_set[queryno] = set([docno])
    with open (QREL_ADHOC, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            items = line.split()
            queryno, docno = int(items[0]), items[2]
            if queryno in union_set.keys():
                union_set[queryno].add(docno)
    return union_set
union_set = okapi_qrel_union(QREL_ADHOC, OKAPI_BM_TOP1000)

# Map query with texts before feeding into CountVectorizer

In [3]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
snowball = SnowballStemmer('english')

def is_not_number(token):
    try:
        float(token)
    except ValueError:
        return True
    return False

def retrieve_texts(union_set, index):
    query_text_map, query_doc_map = {}, {}
    for queryno, docs in tqdm(union_set.items()):
        texts = []
        counter = 0
        for i, id in enumerate(docs):
            if queryno in query_doc_map:
                query_doc_map[queryno].append(id)
            else:
                query_doc_map[queryno] = [id]
            counter += 1
            res = es.termvectors(index=index, id=id)['term_vectors']
            terms = [term for term in res['text']['terms'] if is_not_number(term)]
            text = ' '.join(terms)
            texts.append(text)
            # print(text)
        query_text_map[queryno] = texts
    return query_doc_map, query_text_map

query_doc_map, query_text_map = retrieve_texts(union_set, 'ap_data')

100%|██████████| 25/25 [01:21<00:00,  3.24s/it]


# Get top k topics per query

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer(min_df=0.02, max_df=0.95, stop_words='english', analyzer='word')

def top_k_words(k, model, feature_names):
    top_list, top_k_list = [], []
    for id, topic in enumerate(model.components_):
        topic_list = list(topic)
        max_index = topic_list.index(max(topic_list))
        top_list.append((topic_list[max_index], features[max_index]))
    top_list = sorted(top_list, reverse=True)
    for tuple in [item for item in top_list if is_not_number(item[1])]:
        word = tuple[1]
        if word not in top_k_list:
            top_k_list.append(word)
        if len(top_k_list) == k:
            break
    return top_k_list

with open('top_query_topics.txt', 'w') as file:
    for query, texts in tqdm(query_text_map.items(), desc='retreving top 20 words per query'):
        sparse_matrix = vectorizer.fit_transform(texts)
        lda = LatentDirichletAllocation(n_components = sparse_matrix.shape[0], max_iter=5,
                                    learning_method='online',
                                    learning_offset=50., random_state=0)
        features = vectorizer.get_feature_names()
        lda.fit(sparse_matrix)
        file.write('{}: {}\n'.format(str(query), str(top_k_words(20, lda, features))))

100%|██████████| 50/50 [17:47<00:00, 21.35s/it]


# Get top k topics over each document

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

vectorizer = CountVectorizer(stop_words='english', analyzer='word')

with open ('top_topics_per_doc.txt', 'w', encoding='ISO-8859-1') as f:
    for queryno in tqdm(query_text_map, desc='getting top topics for each document...'):
        index = 0
        for text in query_text_map[queryno]:
            temp = []
            sparse_matrix = vectorizer.fit_transform([text])
            lda = LatentDirichletAllocation(n_components = sparse_matrix.shape[1], max_iter=5,
                                        learning_method='online',
                                        learning_offset=50., random_state=0)
            features = vectorizer.get_feature_names()
            disx)
            query_doc_map[queryno][index]
            topic = lda.components_[0]
            top_topic_words = topic.argsort()[-10:]

            for i in top_topic_words:
                if features[i].isalpha() and is_not_number(features[i]):
                    temp.append(features[i])

            f.write('{}/{}/{}\n'.format(str(queryno), query_doc_map[queryno][index], str(temp)))

            index += 1

erword_bound)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:790: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:790: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:790: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:790: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:790: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)
/Library/Fr

# Part B

In [1]:
from elasticsearch import Elasticsearch, helpers
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import string

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
snowball = SnowballStemmer('english')

def es_ids(index):
    a=helpers.scan(es,query={"query":{"match_all": {}}},scroll='1m',index=index)
    return [aa['_id'] for aa in a]

def is_not_number(token):
    try:
        float(token)
    except ValueError:
        return True
    return False

def build_feature_texts(all_docs):
    texts = []
    doc_map = {}
    for index, id in tqdm(enumerate(all_docs), desc='getting all feature texts'):
        doc_map[index] = id
        res = es.search(index='ap_data', body={"query":{"match":{"_id":id}}})
        raw_text = res['hits']['hits'][0]['_source']['text']
        tokens = word_tokenize(raw_text.translate(translator))
        text = ' '.join([snowball.stem(token) for token in tokens if is_not_number(token)])
        texts.append(text)
    return doc_map, texts

getting all feature texts: 84678it [15:54, 88.72it/s]


## pickle list of texts

In [2]:
import pickle

all_docs =es_ids('ap_data')
doc_map, texts = build_feature_texts(all_docs)

with open('texts.pkl', 'wb') as fp:
    pickle.dump(texts, fp)

with open('doc_map.pkl', 'wb') as fp:
    pickle.dump(doc_map, fp)

In [1]:
import pickle

with open('texts.pkl', 'rb') as f:
    texts = pickle.load(f)

with open('doc_map.pkl', 'rb') as f:
    doc_map = pickle.load(f)

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

T = 200

vectorizer = CountVectorizer(min_df=0.02, max_df=0.95, stop_words='english', analyzer='word')
sparse_matrix = vectorizer.fit_transform(texts)
lda = LatentDirichletAllocation(n_components = T, max_iter=5,
                            learning_method='online',
                            learning_offset=50., random_state=0)

features = vectorizer.get_feature_names()
distribution = lda.fit_transform(sparse_matrix)
# top_topic_words = distribution[0].argsort()[-10:]

In [4]:
def print_top_words(model, feature_names, n_top_words):
    counter = 0
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % (topic_idx + 1)
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print_top_words(lda, features, 10)

Topic #1: promot cocain larg low defeat music israel circuit daili english
Topic #2: report said offici news say sourc agenc quot told today
Topic #3: play music star perform record movi seri stage role audienc
Topic #4: store shop chain busi retail owner cloth custom wear item
Topic #5: bush american oper state presid initi octob critic unit nation
Topic #6: book king write work world wrote god religi year life
Topic #7: question point view chang answer time clear believ cours fact
Topic #8: recov fraud intent agricultur los houston juri sponsor key lawyer
Topic #9: report said today citi govern nation demonstr anti soviet protest
Topic #10: said state law decis court speech case word languag peopl
Topic #11: sale week sell sold buy year said number ticket purchas
Topic #12: societi welcom defens debt measur newspap cold concentr tom unchang
Topic #13: base action michigan philadelphia ask player reflect divid speed princip
Topic #14: program plan propos approv packag new measur requi

# K-Means Clustering

In [5]:
from sklearn.cluster import KMeans

K = 25

clusters = {}
kMeans = KMeans(n_clusters = K)
kMeans.fit(distribution)

for labelIndex in range(0, len(kMeans.labels_)):
    if kMeans.labels_[labelIndex] not in clusters:
        clusters[kMeans.labels_[labelIndex]] = set()
    clusters[kMeans.labels_[labelIndex]].add(doc_map[labelIndex])


In [8]:
QREL_FILE = '../HW1/trec_eval/qrels.adhoc.51-100.AP89.txt'

def retrieve_qrel(QREL_FILE):
    counter = 0
    queryno_list = [59, 77, 94, 85, 95, 91, 56, 71, 64, 62, 93, 99, 58, 54, 87, 100, 89, 61, 68, 63, 57, 97, 98, 60, 80]
    print(len(queryno_list))
    with open (QREL_FILE, 'r', encoding='ISO-8859-1') as f:
        qrel_dict = {}
        rel_docs = []
        for line in f:
            items = line.split()
            queryno, docno, relevance = int(items[0]), items[2], float(items[3])
            if relevance == 1 and queryno in queryno_list:
                counter += 1
                rel_docs.append(docno)
                if queryno in qrel_dict:
                    qrel_dict[queryno].append(docno)
                else:  
                    qrel_dict[queryno] = [docno]
    print(counter)
    return rel_docs
    
rel_docs = retrieve_qrel(QREL_FILE)

25
1832


In [9]:
import numpy as np

def rel_docs_C_2(docs):
    for i, j in zip(*np.triu_indices(len(docs), 1)):
        yield docs[i], docs[j]

pairs = list(rel_docs_C_2(rel_docs))

In [10]:
def SQ(doc1, doc2):
    for queryno, doc_list in query_doc_map.items():
        if doc1 in doc_list and doc2 in doc_list:
            return True
    return False

def SC(doc1, doc2):
    for label, doc_list in clusters.items():
        if doc1 in doc_list and doc2 in doc_list:
            return True
    return False

In [18]:
from tqdm import tqdm

SQSC, SQDC, DQSC, DQDC = 0, 0, 0, 0

counter = 0

for pair in tqdm(pairs, desc='inspecting all the combos'):
    counter += 1
    doc1, doc2 = pair[0], pair[1]
    if SQ(doc1, doc2) and SC(doc1, doc2):
        SQSC += 1
    elif SQ(doc1, doc2) and not SC(doc1, doc2):
        SQDC += 1
    elif not SQ(doc1, doc2) and SC(doc1, doc2):
        DQSC += 1
    elif not SQ(doc1, doc2) and not SC(doc1, doc2):
        DQDC += 1
print(SQSC, SQDC, DQSC, DQDC)

inspecting all the combos: 100%|██████████| 1677196/1677196 [57:51<00:00, 483.20it/s]70653 153606 105334 1347603



In [22]:
accuracy = (SQSC + DQDC) / (SQDC + DQSC + SQSC + DQDC) * 100
print(accuracy, '%')

84.5611365636455 %
